TESTING SAME IMPLEMENTATION WITH SKLEARN DECISION TREE CLASSIFIER

In [ ]:
#imports
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#create data
math_data = pd.read_csv('student-mat.csv', delimiter = ';')
port_data = pd.read_csv('student-por.csv', delimiter = ';')
data = pd.concat([math_data, port_data]).reset_index()

weighted_grades = []
for row in data.index:
  first_half = data['G1'][row]
  second_half = data['G2'][row]
  final = data['G3'][row]
  weighted_total = (first_half*0.25+second_half*0.25+final*0.5)*5/100

  if weighted_total >= 0.6:
    weighted_grades.append(1)
  else:
    weighted_grades.append(0)
data['weighted_grade'] = weighted_grades


#dropping irrelevant columns like schoos, age, etc.
#columns are selected based on information gain after first split, if gain < 0.001, then consider it irrelevant
data.drop(['index','famsize','Pstatus','schoolsup','famsup','school','age',
           'guardian','nursery','romantic','activities','freetime','goout',
           'Dalc','Walc','Mjob','Fjob','reason','traveltime','G1','G2','G3'],
          axis=1,inplace=True)

data.head()

,sex,address,Medu,Fedu,studytime,failures,paid,higher,internet,famrel,health,absences,weighted_grade
0,F,U,4,4,2,0,no,yes,no,4,3,6,0
1,F,U,1,1,2,0,no,yes,yes,5,3,4,0
2,F,U,1,1,2,3,yes,yes,yes,4,3,10,0
3,F,U,4,2,3,0,yes,yes,yes,3,5,2,1
4,F,U,3,3,2,0,yes,yes,no,4,5,4,0


In [ ]:
X = data.copy()
#whatever the pass column is called in the csv
Y = data['weighted_grade']

#roughly 80-20 split between training and testing data
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 0.2)

clf = DecisionTreeClassifier()
clf = clf.fit(X, Y)

ValueError: ignored

In [ ]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict([[1, 2]])

array([1])

In [ ]:
                                                  ##################################################################################
                                                  # REMEMBER TO RELOAD THE TWO CSV DATA BEFORE RUNNING THE CODE, THEY COULD EXPIRE #
                                                  ##################################################################################
import pandas as pd
import numpy as np
import math
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

#increase maximum recursion depth
import sys
sys.setrecursionlimit(10000)

#load and concatenate data
math_data = pd.read_csv('student-mat.csv', delimiter = ';')
port_data = pd.read_csv('student-por.csv', delimiter = ';')
data = pd.concat([math_data, port_data]).reset_index()

#apply 25% weight on G1 & G2 grades and 50% weight on G3(final) grade
#labeling students as 'P'(pass) if students' weighted grade is > 60% or 'F'(fail) otherwise
weighted_grades = []
for row in data.index:
  first_half = data['G1'][row]
  second_half = data['G2'][row]
  final = data['G3'][row]
  weighted_total = (first_half*0.25+second_half*0.25+final*0.5)*5/100

  if weighted_total >= 0.6:
    weighted_grades.append('P')
  else:
    weighted_grades.append('F')
data['weighted_grade'] = weighted_grades


#dropping irrelevant columns like schoos, age, etc.
#columns are selected based on information gain after first split, if gain < 0.001, then consider it irrelevant
data.drop(['index','famsize','Pstatus','schoolsup','famsup','school','age',
           'guardian','nursery','romantic','activities','freetime','goout',
           'Dalc','Walc','Mjob','Fjob','reason','traveltime','G1','G2','G3'],
          axis=1,inplace=True)

all_data = data.copy()
data = all_data.sample(n=850)
test_data = all_data.sample(n = 200)

#data entropy calculation function
def entropy(input_data):
  '''
  @param input_data: pandas dataframe used to calculate entropy

  @return the entropy of data
  '''
  passes = 0
  failed = 0
  for row in input_data.index:
    if input_data['weighted_grade'][row] == 'P':
      passes += 1
    else:
      failed += 1

  if (passes == 0):
    return - (failed/(passes+failed))*math.log2(failed/(passes+failed))
  elif (failed == 0):
    return -(passes/(passes+failed))*math.log2(passes/(passes+failed))
  else:
    return -(passes/(passes+failed))*math.log2(passes/(passes+failed)) - (failed/(passes+failed))*math.log2(failed/(passes+failed))


#information gain calculation function
def calculateGain(input_data, attribute):
  '''
  @param input_data: pandas dataframe used to calculate information gain
  @param attribute: data splitting attribute

  @return the information gain of the data set splitting by attribute
  '''
  if (attribute == 'sex'):
    data_left = input_data[input_data[attribute] == 'M'].reset_index()
    data_right = input_data[input_data[attribute] == 'F'].reset_index()
  elif (attribute == 'address'):
    data_left = input_data[input_data[attribute] == 'U'].reset_index()
    data_right = input_data[input_data[attribute] == 'R'].reset_index()
  elif (attribute == 'Medu'):
    data_left = input_data[input_data[attribute] <= 3].reset_index()
    data_right = input_data[input_data[attribute] > 3].reset_index()
  elif (attribute == 'Fedu'):
    data_left = input_data[input_data[attribute] <= 3].reset_index()
    data_right = input_data[input_data[attribute] > 3].reset_index()
  elif (attribute == 'studytime'):
    data_left = input_data[input_data[attribute] < 3].reset_index()
    data_right = input_data[input_data[attribute] >= 3].reset_index()
  elif (attribute == 'failures'):
    data_left = input_data[input_data[attribute] < 1].reset_index()
    data_right = input_data[input_data[attribute] >= 1].reset_index()
  elif (attribute == 'paid'):
    data_left = input_data[input_data[attribute] == 'yes'].reset_index()
    data_right = input_data[input_data[attribute] == 'no'].reset_index()
  elif (attribute == 'higher'):
    data_left = input_data[input_data[attribute] == 'yes'].reset_index()
    data_right = input_data[input_data[attribute] == 'no'].reset_index()
  elif (attribute == 'internet'):
    data_left = input_data[input_data[attribute] == 'yes'].reset_index()
    data_right = input_data[input_data[attribute] == 'no'].reset_index()
  elif (attribute == 'famrel'):
    data_left = input_data[input_data[attribute] <= 3].reset_index()
    data_right = input_data[input_data[attribute] > 3].reset_index()
  elif (attribute == 'health'):
    data_left = input_data[input_data[attribute] <= 3].reset_index()
    data_right = input_data[input_data[attribute] > 3].reset_index()
  elif (attribute == 'absences'):
    data_left = input_data[input_data[attribute] <= 5].reset_index()
    data_right = input_data[input_data[attribute] > 5].reset_index()

  #if either left or right part after splitting is empty, quit
  if (data_left.empty or data_right.empty):
    return None, None, 10

  gain = entropy(input_data)-(len(data_left)/len(input_data))*entropy(data_left)-(len(data_right)/len(input_data))*entropy(data_right)
  return data_left, data_right, gain



#defining tree node class
#A tree node has:
#    its left node
#    its right node
#    dataframe it contains
#    splitter of itself if it has children
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data
      self.split = None

#setting the original datafram as the root node
root_node = Node(data)

#recursion function to construct the binary decision tree
def recursionTree(node):
  #base cases: no attribute to split or less than 5% error rate
  if (node.data.shape[1] == 1):
    return 0
  passes = 0
  failed = 0
  for row in node.data.index:
    if node.data['weighted_grade'][row] == 'P':
      passes += 1
    else:
      failed += 1
  if ((passes / (passes + failed) >= 0.95) or (failed / (passes + failed) >= 0.95)):
    return 0

  #recursion part
  #for loop to obtain information gain for all columns
  columns = []  
  gains = []
  for column in node.data:
    if (column == 'weighted_grade'):
      break
    #if all the rows have the same value in a certain column, drop that column
    if (node.data[column] == node.data[column][0]).all():
      node.data.drop([column],axis=1,inplace=True)
      continue
    dl, dr, gain = calculateGain(node.data, column)
    #if gain = 10, that means splitting has been forfeited, then ignore the column
    if (gain == 10):
      continue
    #if split and calculation successful, record information gain the corresponding column name
    gains.append(gain)
    columns.append(column)

  #if no column has been chosen for splitting, then reach base case
  if (len(gains) == 0):
    return 0
  
  #choose the column with the most information gain
  gains = np.array(gains)
  index = np.flip(np.argsort(gains))
  split_attri = columns[index[0]]

  #set resulting left,right children and splitter
  dl, dr, gain = calculateGain(node.data, split_attri)
  dl.drop(['index',split_attri], axis=1,inplace=True)
  dr.drop(['index',split_attri], axis=1,inplace=True)
  node.left=Node(dl)
  node.right=Node(dr)
  node.split=split_attri

  return recursionTree(node.left) + recursionTree(node.right)

recursionTree(root_node)

0

In [ ]:
#The final classifier after applying decision tree algorithm
def studentGradeClassifier(root, sex,address,Medu,Fedu,studytime,failures,paid,higher,internet,famrel,health,absences):
  '''
  @param sex: student's sex (binary: "F" - female or "M" - male)
  @param address: student's home address type (binary: "U" - urban or "R" - rural)
  @param Medu: mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
  @param Fedu: father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
  @param studytime: weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
  @param failures: number of past class failures (numeric: n if 1<=n<3, else 4)
  @param paid: extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
  @param higher: wants to take higher education (binary: yes or no)
  @param internet: Internet access at home (binary: yes or no)
  @param famrel: quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
  @param health: current health status (numeric: from 1 - very bad to 5 - very good)
  @param absences: number of school absences (numeric: from 0 to 93)

  @return the classification of the student, whether he'll P(pass) or F(fail)
  '''
  node = root
  while (node.split != None):
    att = node.split
    if (att == 'sex'):
      if (sex == 'M'):
        node = node.left
      else:
        node = node.right
    elif (att == 'address'):
      if (address == 'U'):
        node = node.left
      else:
        node = node.right
    elif (att == 'Medu'):
      if (Medu <= 3):
        node = node.left
      else:
        node = node.right
    elif (att == 'Fedu'):
      if (Fedu <= 3):
        node = node.left
      else:
        node = node.right
    elif (att == 'studytime'):
      if (studytime < 3):
        node = node.left
      else:
        node = node.right
    elif (att == 'failures'):
      if (failures< 1):
        node = node.left
      else:
        node = node.right
    elif (att == 'paid'):
      if (paid == 'yes'):
        node = node.left
      else:
        node = node.right
    elif (att == 'higher'):
      if (higher == 'yes'):
        node = node.left
      else:
        node = node.right
    elif (att == 'internet'):
      if (internet == 'yes'):
        node = node.left
      else:
        node = node.right
    elif (att == 'famrel'):
      if (famrel <= 3):
        node = node.left
      else:
        node = node.right
    elif (att == 'health'):
      if (health <= 3):
        node = node.left
      else:
        node = node.right
    elif (att == 'absences'):
      if (absences <= 5):
        node = node.left
      else:
        node = node.right

  ps = 0
  fa = 0
  for row in node.data.index:
    if node.data['weighted_grade'][row] == 'P':
      ps += 1
    else:
      fa += 1
  if (ps >= fa):
    return 'P'
  else:
    return 'F'

#calculate the accuracy of the classifier
total_correct = 0
total_num = 0
data = test_data
for row in data.index:
  result = studentGradeClassifier(root_node, data['sex'][row],data['address'][row],data['Medu'][row],
                                  data['Fedu'][row],data['studytime'][row],data['failures'][row],
                                  data['paid'][row],data['higher'][row],data['internet'][row],
                                  data['famrel'][row],data['health'][row],data['absences'][row])
  if result == data['weighted_grade'][row]:
    total_correct += 1
  total_num += 1
print('Total Correct Classification: ', total_correct, '\nTotal data: ', 
      total_num, '\nModel Accuracy: ', total_correct/total_num*100, '%')

Total Correct Classification:  166 
Total data:  200 
Model Accuracy:  83.0 %


In [ ]:
#https://leetcode.com/discuss/interview-question/1954462/pretty-printing-binary-trees-in-python-for-debugging
#function for printing pretty binary tree
from IPython.core.magics.execution import pstats
def display(root):
    lines, *_ = _display_aux(root)
    for line in lines:
        print(line)

def _display_aux(self):
    """Returns list of strings, width, height, and horizontal coordinate of the root."""
    # No child.
    if self.right is None and self.left is None:
        line = '%s' % self.split
        if (line == 'None'):
          dt = self.data
          ps = 0
          fa = 0
          for row in dt.index:
            if dt['weighted_grade'][row] == 'P':
              ps += 1
            else:
              fa += 1
          if (ps >= fa):
            line = 'P'
          else:
            line = 'F'
        width = len(line)
        height = 1
        middle = width // 2
        return [line], width, height, middle

    # Only left child.
    if self.right is None:
        lines, n, p, x = _display_aux(self.left)
        s = '%s' % self.split

        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
        second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
        shifted_lines = [line + u * ' ' for line in lines]
        return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

    # Only right child.
    if self.left is None:
        lines, n, p, x = _display_aux(self.right)
        s = '%s' % self.split

        u = len(s)
        first_line = s + x * '_' + (n - x) * ' '
        second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
        shifted_lines = [u * ' ' + line for line in lines]
        return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

    # Two children.
    left, n, p, x = _display_aux(self.left)
    right, m, q, y = _display_aux(self.right)
    s = '%s' % self.split

    u = len(s)
    first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
    second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
    if p < q:
        left += [n * ' '] * (q - p)
    elif q < p:
        right += [m * ' '] * (p - q)
    zipped_lines = zip(left, right)
    lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
    return lines, n + m + u, max(p, q) + 2, n + u // 2

display(root_node)